<a href="https://colab.research.google.com/github/Akshay-Kumar-Arya/Identify_the_sentiments/blob/master/bert_GRU_sequence_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRU sequence model using Bert vectors

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import tensorflow as tf
from sklearn import svm
from sklearn.metrics import f1_score
pd.set_option('display.max_colwidth', 200)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Data path
training_data_path = "/content/gdrive/My Drive/Identify_the_sentiments/train.csv"
test_data_path =  "/content/gdrive/My Drive/Identify_the_sentiments/test.csv"

save_path = "/content/gdrive/My Drive/Identify_the_sentiments/"

In [3]:
# reading data from csv
train_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

print (train_data.shape, test_data.shape)

(7920, 3) (1953, 2)


In [4]:
# import bert embeddings for tweets
train_file = open(save_path + "bert_train.pickle", "rb")
pool_embs_train, all_embs_train = pickle.load(train_file)

test_file = open(save_path + "bert_test.pickle", "rb")
pool_embs_test, all_embs_test = pickle.load(test_file)

In [5]:
# splitting training and validation data
xtrain, xvalid, ytrain, yvalid = train_test_split(all_embs_train, 
                                                  train_data['label'],
                                                  random_state=42, 
                                                  test_size=0.2)
print (xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(6336, 128, 768) (1584, 128, 768) (6336,) (1584,)


In [6]:
#first
model = tf.keras.models.Sequential([
                                    tf.keras.layers.GRU(units=8, input_shape=(128, 768)),
                                    tf.keras.layers.Dense(units=1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], optimizer='adam')
history = model.fit(xtrain, ytrain, validation_data=(xvalid, yvalid), epochs=5, batch_size=32)

Epoch 1/5
198/198 [==============================] - 8s 39ms/step - loss: 0.3128 - precision: 0.7579 - recall: 0.6813 - val_loss: 0.2632 - val_precision: 0.7485 - val_recall: 0.8889
Epoch 2/5
198/198 [==============================] - 7s 36ms/step - loss: 0.2430 - precision: 0.7787 - recall: 0.8457 - val_loss: 0.2538 - val_precision: 0.8090 - val_recall: 0.7940
Epoch 3/5
198/198 [==============================] - 7s 36ms/step - loss: 0.2236 - precision: 0.7954 - recall: 0.8657 - val_loss: 0.2554 - val_precision: 0.8155 - val_recall: 0.7778
Epoch 4/5
198/198 [==============================] - 7s 36ms/step - loss: 0.2089 - precision: 0.8020 - recall: 0.8896 - val_loss: 0.2590 - val_precision: 0.8158 - val_recall: 0.7894
Epoch 5/5
198/198 [==============================] - 7s 36ms/step - loss: 0.1985 - precision: 0.8097 - recall: 0.9021 - val_loss: 0.2386 - val_precision: 0.7755 - val_recall: 0.8796


In [9]:
# prediction on validation dataset
preds_valid = model.predict(xvalid)
#print (f1_score(yvalid, preds_valid))
preds_valid[preds_valid < 0.5] = 0
preds_valid[preds_valid >= 0.5] = 1

In [10]:
# first
f1_score(yvalid, preds_valid)

0.824295010845987

In [11]:
# prediction on test dataset
import numpy as np
preds_test = model.predict(all_embs_test)
preds_test[preds_test < 0.5] = 0
preds_test[preds_test >= 0.5] = 1

preds_test = np.squeeze(preds_test)

# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_rnn_bert.csv", index=False)
# Your score for this submission is : 0.8989105276853336.
# Your score for this submission is : 0.8916512076117228.
